In [2]:
from collections import defaultdict
from gensim import corpora
import pandas as pd

In [80]:
df = pd.read_csv(r"C:\Users\Philippa\Documents\GitHub\crystalline-mining\extracted_pdf_info.csv")

df.head()

,pdf_text,metadata
0,['NATURAL CALCIUM CARBONATE FOR BIOMEDICAL APP...,"{'Author': 'ismail - [2010]', 'Creator': 'Micr..."
1,['Proceedings of Machine Learning Research LEA...,"{'Author': '', 'CreationDate': 'D:202404290022..."
2,['Preprint Advances in Chemical Physics Vol. 1...,"{'CreationDate': 'D:20060424095709Z', 'ModDate..."
3,['Enhancing Drug-Drug Interaction Extraction f...,"{'Author': '', 'CreationDate': 'D:201805160029..."
4,['BERTChem-DDI : Improved Drug-Drug Interactio...,"{'Author': '', 'CreationDate': 'D:202012230158..."


In [81]:
from gensim import corpora, models
from gensim.corpora.dictionary import Dictionary

In [82]:
# create the corpus to be used in LDA model
docs = df["pdf_text"].values.copy()

In [83]:
from nltk.tokenize import RegexpTokenizer

# convert text to lowercase, split the documents into tokens
tokenizer = RegexpTokenizer(r"\w+")
for idx in range(len(docs)):
    docs[idx] = docs[idx].lower()
    docs[idx] = tokenizer.tokenize(docs[idx])
    
# remove numbers but not words that contain numbers
docs = [[token for token in doc if not token.isnumeric()] for doc in docs]

# remove words that are only one character
docs = [[token for token in doc if len(token) > 1] for doc in docs]

In [84]:
# compute bigrams (i.e. phrases like machine_learning)
from gensim.models import Phrases

# Add bigrams and trigrams to docs (only ones that appear 20 times or more)
bigram = Phrases(docs, min_count=20)
for idx in range(len(docs)):
    for token in bigram[docs[idx]]:
        if "_" in token:
            # token is a bigram, add to document
            docs[idx].append(token)

In [85]:
# remove rare and common tokens
from gensim.corpora import Dictionary

# create a dictionary representation of the documents
dictionary = Dictionary(docs)

# filter out words that occur less than 10 documents, or more than 50% of the documents
dictionary.filter_extremes(no_below=10, no_above=0.5)

In [86]:
# Bag-of-words representation of the documents.
corpus = [dictionary.doc2bow(doc) for doc in docs]

In [87]:
from gensim.models import LdaModel

lda = LdaModel(
    corpus=corpus,
    id2word=dictionary,
    num_topics=10,
    passes=20,
    iterations=400,
    alpha="auto",
    eta="auto",
    eval_every=None
)

In [88]:
lda.show_topics(num_topics=10, num_words=10)

[(0,
  '0.011*"3d" + 0.009*"atom" + 0.007*"property" + 0.006*"vol" + 0.006*"drug_design" + 0.005*"generative" + 0.005*"module" + 0.004*"crystal" + 0.004*"vol_pp" + 0.004*"image"'),
 (1,
  '0.022*"ddi" + 0.009*"embeddings" + 0.008*"relation" + 0.007*"ddis" + 0.007*"extraction" + 0.007*"text" + 0.006*"heterogeneous" + 0.006*"patient" + 0.005*"drugbank" + 0.004*"drug_pairs"'),
 (2,
  '0.009*"doi" + 0.007*"editing" + 0.007*"binding" + 0.006*"targets" + 0.006*"product" + 0.006*"gnn" + 0.005*"identified" + 0.005*"base" + 0.005*"peptide" + 0.005*"drug_target"'),
 (3,
  '0.924*"cid_cid" + 0.034*"dna" + 0.001*"pdf" + 0.001*"3d_structures" + 0.000*"proximity" + 0.000*"hybridization" + 0.000*"china" + 0.000*"file" + 0.000*"www" + 0.000*"http"'),
 (4,
  '0.035*"cells" + 0.027*"si" + 0.024*"cancer" + 0.011*"targeted" + 0.010*"cancer_cells" + 0.010*"therapy" + 0.008*"resistance" + 0.008*"resistant" + 0.008*"growth" + 0.007*"tumor"'),
 (5,
  '0.022*"response" + 0.010*"dose" + 0.010*"domain" + 0.008*"

### SKlearn tokenization method

In [ ]:
# from sklearn.feature_extraction.text import CountVectorizer

# # after pre-processing text, begin process for automatic topic selection
# # Join tokens into text
# preprocessed_texts = [" ".join(doc) for doc in docs]

# # 3. Vectorise
# vectorizer = CountVectorizer(
#     lowercase=True,       # already lowercased
#     token_pattern=r"(?u)\b[a-zA-Z]{2,}\b", 
#     stop_words="english"        # already removed unwanted tokens
# )
# word_counts = vectorizer.fit_transform(preprocessed_texts)

# from sklearn.decomposition import LatentDirichletAllocation
# lda = LatentDirichletAllocation(n_components=9,
#                                 random_state=42)
# lda.fit(word_counts)

LatentDirichletAllocation(n_components=9, random_state=42)

In [ ]:
# feature_names = vectorizer.get_feature_names_out()
# for idx, topic in enumerate(lda.components_):
#     top = topic.argsort()[::-1][:10]
#     print(f"Topic {idx}:", [feature_names[i] for i in top])

Topic 0: ['drug', 'protein', 'target', 'graph', 'based', 'prediction', 'molecular', 'learning', 'binding', 'drugs']
Topic 1: ['fig', 'al', 'layer', 'structures', 'materials', 'chem', 'high', 'strain', 'et', 'different']
Topic 2: ['al', 'pegda', 'et', 'pegmema', 'mof', 'flow', 'fig', 'structure', 'printed', 'mofflow']
Topic 3: ['cid', 'dna', 'origami', 'plasmonic', 'nanostructures', 'strands', 'dietz', 'scaffold', 'stranded', 'staple']
Topic 4: ['et', 'al', 'network', 'drug', 'protein', 'networks', 'disease', 'target', 'analysis', 'related']
Topic 5: ['drug', 'drugs', 'model', 'graph', 'al', 'data', 'et', 'prediction', 'learning', 'based']
Topic 6: ['si', 'li', 'nm', 'phase', 'fig', 'vesicles', 'figure', 'pores', 'pc', 'silicon']
Topic 7: ['drug', 'cells', 'cancer', 'model', 'data', 'release', 'product', 'drugs', 'targeted', 'et']
Topic 8: ['drug', 'information', 'model', 'task', 'data', 'extraction', 'text', 'representation', 'relation', 'molecular']


In [50]:
print("Number of unique tokens: %d" % len(dictionary))
print("Number of documents: %d" % len(docs))

Number of unique tokens: 3742
Number of documents: 99


### Creating Topic Labels 

In [89]:
topic_labels = {
    0: "3D Generative Drug Design Module",
    1: "Drug–Drug Interaction Embedding Extraction",
    2: "GNN-Based Drug-Target Binding Prediction",
    3: "Compound ID Retrieval & 3D Structural Data",
    4: "siRNA Cancer Cell Therapy & Resistance",
    5: "Bayesian Tumor Dose–Response Analysis",
    6: "Protein Targets in Cancer Signaling & Binding",
    7: "Cancer Gene Cell-Line Drug Combination Safety",
    8: "Drug-Target Binding Affinity via SMILES & Sequences",
    9: "Material Release Profiles & Surface Chemistry"
}

### Document-Topic Distributions

In [90]:
doc_topics = [lda.get_document_topics(bow) for bow in corpus]

dominant_topics = [max(dist, key=lambda x: x[1])[0] for dist in doc_topics]

Filter or group papers by theme

In [91]:
df["dominant_topic"] = dominant_topics

### Evaluate Model Quality

In [95]:
from gensim.models import CoherenceModel

cm = CoherenceModel(model=lda, texts=docs, dictionary=dictionary, coherence='c_v')
print("Coherence:", cm.get_coherence())
print("Perplexity:", lda.log_perplexity(corpus))

Coherence: 0.48528173277639136
Perplexity: -7.172655402390722


### Visualise the Topics

In [98]:
import pyLDAvis
import pyLDAvis.gensim_models

vis = pyLDAvis.gensim_models.prepare(lda, corpus, dictionary)

# Enable the notebook renderer
pyLDAvis.enable_notebook()

vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.070271 -0.145008       1        1  17.145074
9      0.037857  0.184541       2        1  13.973074
7      0.072737 -0.042104       3        1  13.821514
6      0.054292  0.043328       4        1  12.385249
8      0.071834 -0.133787       5        1  11.738953
0      0.075154 -0.075909       6        1   8.780819
2      0.076310 -0.099611       7        1   8.166190
3     -0.531869 -0.032237       8        1   5.311064
4      0.027698  0.225692       9        1   4.629769
5      0.045716  0.075095      10        1   4.048292, topic_info=         Term          Freq         Total Category  logprob  loglift
1068  cid_cid  12632.000000  12632.000000  Default  30.0000  30.0000
2125      ddi    993.000000    993.000000  Default  29.0000  29.0000
119    cancer   1218.000000   1218.000000  Default  28.0000  28.0000
1735      dna    590.000000    590.000000  Default  27.0000  27.0000
142     cells    623.000000    623.000000  Default  26.0000  26.0000
...       ...           ...           ...      ...      ...      ...
767    sample     54.872831    299.526115  Topic10  -5.2395   1.5097
1856   ligand     54.925140    425.526485  Topic10  -5.2385   1.1595
119    cancer     76.872558   1218.388133  Topic10  -4.9024   0.4437
804      site     39.645576    203.254374  Topic10  -5.5645   1.5724
142     cells     40.931329    623.822014  Topic10  -5.5326   0.4829

[551 rows x 6 columns], token_table=      Topic      Freq Term
term                      
2883      1  0.155920   1b
2883      3  0.051973   1b
2883      5  0.051973   1b
2883      9  0.675651   1b
3377      3  0.173512   1c
...     ...       ...  ...
977      10  0.028779  you
3258      4  0.053970  yue
3258      5  0.755587  yue
3258      7  0.107941  yue
3258      9  0.107941  yue

[2562 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 10, 8, 7, 9, 1, 3, 4, 5, 6])